# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821



## Will work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Writing the Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Requirement

### Will Create queries to answer/respond to the following;

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Creating Tables to fulfill the above requirements

### Table 1 - Music_apphistory Table
This table will be designed to store the app history related to music, specifically information about songs played in different sessions. 

In [9]:
query = "CREATE TABLE IF NOT EXISTS music_apphistory "
query = query + "(artist text, song_title text, sessionId int, song_length float, itemInSession int, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
         

The primary key (sessionId, itemInSession) is chosen to enable efficient querying based on the session and the order of items within a session. By including sessionId in the primary key, we can retrieve all the interactions within a specific session. The itemInSession column helps preserve the order of items within a session.

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_apphistory (artist, song_title, sessionId, song_length, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query,(line[0], line[9], int(line[8]), float(line[5]), int(line[3])))      

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
query = "select artist, song_title, song_length from music_apphistory WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Table 2 - Song_apphistory Table
This table is designed to store the app history of users, specifically their interactions with songs. 

In [12]:
query = "CREATE TABLE IF NOT EXISTS song_apphistory "
query = query + "(artist text, userid int, song_title text, sessionId int, firstName text, lastName text, itemInSession int, PRIMARY KEY ((userid, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
       

The primary key ((userid, sessionId), itemInSession) is chosen to enable efficient querying of the table based on user, session, and the order of items within a session. By including both userid and sessionId in the primary key, we can retrieve all the interactions of a specific user within a session. Additionally, the itemInSession column ensures that the order of items is preserved and further refines the uniqueness within a session.

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_apphistory (artist, song_title, userid, sessionId, firstName, lastName, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query,(line[0], line[9], int(line[10]), int(line[8]), line[1], line[4], int(line[3])))      

In [14]:
query = "select artist, song_title, firstName, lastName from song_apphistory WHERE userid = 10 and sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song_title, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Table 3 - Song_listeners Table
This table is designed to store information about listeners who have played songs.

In [15]:
query = "CREATE TABLE IF NOT EXISTS song_listeners "
query = query + "(song_title text, userid int, firstName text, lastName text, PRIMARY KEY (song_title, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)

The primary key (song_title, userid) is chosen based on the assumption that multiple users can listen to the same song, and each user can listen to multiple songs. By making song_title and userid part of the primary key, we can efficiently query the table to retrieve information related to a specific song or a specific user.


In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_listeners (song_title, userid, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))      

In [18]:
query = "SELECT firstName, lastName FROM song_listeners WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.firstname, row.lastname)


Jacqueline Lynch
Tegan Levine
Sara Johnson


### Droping  the tables 

In [18]:
query = "drop table music_apphistory"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table song_apphistory"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table song_listeners"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

For each DROP TABLE statement, the code executes the query using session.execute(query) and assigns the result to the rows variable (which is not used in this code snippet). If an exception occurs during the execution of the query, it is caught and printed out for debugging purposes.

### Closing the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()

### References

- I followed used the exercises performed examples